# Flight Price Analysis

This notebook visualizes flight price trends from the CSV data, plotting datetime on the x-axis and Price_EUR on the y-axis with different colors for each flight route. We'll use Plotly for interactive visualizations.

# Import Required Libraries

Import libraries such as pandas for data manipulation and matplotlib for plotting.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

# Load and Inspect the Dataset

Load the flight_prices.csv file into a pandas DataFrame and inspect the first few rows to understand its structure.

In [2]:
# Load the dataset
file_path = 'flight_prices.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded {file_path}")
    
    # Display the first 5 rows
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
    
    # Display basic information about the dataset
    print("\nDataset information:")
    print(df.info())
    
    # Display summary statistics
    print("\nSummary statistics:")
    print(df.describe())
except FileNotFoundError:
    print(f"Error: File {file_path} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded flight_prices.csv

First 5 rows of the dataset:
         Date      Time              Flight  Price_EUR  Price_BRL  \
0  2025-03-25  11:32:55            SDU->BSB     250.29    1555.31   
1  2025-03-25  11:33:10            BSB->SSA     292.79    1819.40   
2  2025-03-25  11:33:33  ORY->GIG, SSA->ORY    4415.09   27435.04   
3  2025-03-25  11:47:05            BSB->SSA     292.79    1819.40   
4  2025-03-25  11:47:28  ORY->GIG, SSA->ORY    4415.09   27435.04   

   Target_Price  
0          1500  
1          1700  
2         27000  
3          1700  
4         27000  

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          151 non-null    object 
 1   Time          151 non-null    object 
 2   Flight        151 non-null    object 
 3   Price_EUR     151 non-null    float64
 4   Price_BRL     151 no

# Preprocess the Data

Convert the datetime column to a pandas datetime object and ensure the data is sorted by datetime. Identify unique flights for color differentiation.

In [3]:
# Convert datetime column to datetime object
try:
    # Check if 'datetime' column exists
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
    else:
        # Try to find a column that might contain datetime information
        datetime_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        if datetime_cols:
            df['datetime'] = pd.to_datetime(df[datetime_cols[0]])
            print(f"Using {datetime_cols[0]} as datetime column")
        else:
            print("No datetime column found. Please specify the correct column name.")

    # Sort DataFrame by datetime
    df.sort_values('datetime', inplace=True)
    
    # Identify unique flights
    # Checking if columns that might identify flights exist
    flight_id_columns = [col for col in df.columns if 'flight' in col.lower() or 'route' in col.lower() or 'id' in col.lower()]
    
    if flight_id_columns:
        flight_id_col = flight_id_columns[0]
        print(f"Using {flight_id_col} to identify unique flights")
    else:
        # If no obvious flight ID column, we'll check if origin and destination columns exist
        origin_dest_cols = [col for col in df.columns if 'origin' in col.lower() or 'destination' in col.lower() 
                           or 'from' in col.lower() or 'to' in col.lower()]
        
        if len(origin_dest_cols) >= 2:
            # Create a flight identifier from origin and destination
            flight_id_col = 'flight_id'
            df[flight_id_col] = df[origin_dest_cols[0]] + '-' + df[origin_dest_cols[1]]
            print(f"Created {flight_id_col} from {origin_dest_cols[0]} and {origin_dest_cols[1]}")
        else:
            # If we can't find appropriate columns, warn the user
            print("Warning: Could not identify flight identifiers. Using row indices as flight identifiers.")
            flight_id_col = 'index'
            df[flight_id_col] = df.index
    
    # Get unique flights
    unique_flights = df[flight_id_col].unique()
    print(f"\nNumber of unique flights: {len(unique_flights)}")
    print(f"Unique flights: {unique_flights[:5]} {'...' if len(unique_flights) > 5 else ''}")

except Exception as e:
    print(f"Error during preprocessing: {e}")

Using Date as datetime column
Using Flight to identify unique flights

Number of unique flights: 3
Unique flights: ['SDU->BSB' 'ORY->GIG, SSA->ORY' 'BSB->SSA'] 


# Plot Flight Prices

Use matplotlib to create a scatter plot with datetime on the x-axis and Price_EUR on the y-axis. Assign a unique color to each flight using a colormap.

In [4]:
# Plot flight prices over time
try:
    # Set up the figure with a larger size
    plt.figure(figsize=(14, 8))
    
    # Check if Price_EUR column exists, if not try to find a price column
    if 'Price_EUR' in df.columns:
        price_col = 'Price_EUR'
    else:
        price_cols = [col for col in df.columns if 'price' in col.lower() or 'cost' in col.lower() or 'fare' in col.lower()]
        if price_cols:
            price_col = price_cols[0]
            print(f"Using {price_col} as price column")
        else:
            price_col = df.select_dtypes(include=[np.number]).columns[0]  # Use the first numeric column
            print(f"No obvious price column found. Using {price_col} as price column")
    
    # Create a colormap
    cmap = plt.cm.get_cmap('viridis', len(unique_flights))
    
    # Plot each flight with a different color
    for i, flight in enumerate(unique_flights):
        flight_data = df[df[flight_id_col] == flight]
        plt.scatter(flight_data['datetime'], flight_data[price_col], 
                   label=f"Flight: {flight}", 
                   color=cmap(i), 
                   alpha=0.7, 
                   s=50)  # s is the marker size
    
    # Formatting the plot
    plt.title('Flight Price Trends Over Time', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel(price_col, fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Format the x-axis to show dates nicely
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gcf().autofmt_xdate()  # Rotate date labels for better readability
    
    # Add legend with smaller font to accommodate many flights
    if len(unique_flights) > 10:
        plt.legend(fontsize=8, loc='upper center', bbox_to_anchor=(0.5, -0.15), 
                  fancybox=True, shadow=True, ncol=5)
    else:
        plt.legend(fontsize=10)
        
    # Show the plot
    plt.tight_layout()
    plt.show()
    
    # Save the figure
    plt.savefig('flight_price_trends.png', dpi=300, bbox_inches='tight')
    print("Plot saved as 'flight_price_trends.png'")
    
except Exception as e:
    print(f"Error during plotting: {e}")

Error during plotting: name 'plt' is not defined


# Additional Analysis (Optional)

Let's add some statistical analysis to better understand the price variations.

In [5]:
# Calculate price statistics by flight
try:
    # Group by flight and calculate statistics
    flight_stats = df.groupby(flight_id_col)[price_col].agg(['mean', 'min', 'max', 'std', 'count'])
    flight_stats = flight_stats.sort_values('mean', ascending=False)
    
    print("Price statistics by flight:")
    print(flight_stats)
    
    # Plot a bar chart of average prices by flight
    plt.figure(figsize=(12, 6))
    flight_stats['mean'].plot(kind='bar', color='skyblue', alpha=0.7)
    plt.title('Average Price by Flight', fontsize=16)
    plt.xlabel('Flight', fontsize=14)
    plt.ylabel(f'Average {price_col}', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', alpha=0.3)
    plt.show()
    
    # Time-based analysis: Calculate average price by month or day of week if data spans a long enough period
    if df['datetime'].max() - df['datetime'].min() > pd.Timedelta(days=30):
        df['month'] = df['datetime'].dt.month_name()
        df['day_of_week'] = df['datetime'].dt.day_name()
        
        # Average price by month
        monthly_avg = df.groupby('month')[price_col].mean().reindex(
            ['January', 'February', 'March', 'April', 'May', 'June', 
             'July', 'August', 'September', 'October', 'November', 'December']
        )
        
        plt.figure(figsize=(12, 6))
        monthly_avg.plot(kind='bar', color='lightgreen', alpha=0.7)
        plt.title('Average Price by Month', fontsize=16)
        plt.xlabel('Month', fontsize=14)
        plt.ylabel(f'Average {price_col}', fontsize=14)
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Average price by day of week
        dow_avg = df.groupby('day_of_week')[price_col].mean().reindex(
            ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        )
        
        plt.figure(figsize=(12, 6))
        dow_avg.plot(kind='bar', color='salmon', alpha=0.7)
        plt.title('Average Price by Day of Week', fontsize=16)
        plt.xlabel('Day of Week', fontsize=14)
        plt.ylabel(f'Average {price_col}', fontsize=14)
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()
    
except Exception as e:
    print(f"Error during additional analysis: {e}")

Error during additional analysis: name 'price_col' is not defined


# Conclusion

Summary of findings from the flight price analysis:

1. We've visualized how flight prices change over time for different flights
2. Identified which flights tend to be more expensive on average
3. Analyzed price trends over months and days of the week (if data permits)

This information can help travelers make more informed decisions about when to book flights to get better prices.

In [6]:
# Load the flight price data
df = pd.read_csv('flight_prices.csv')

# Display the first few rows to understand the data
df.head()

,Date,Time,Flight,Price_EUR,Price_BRL,Target_Price
0,2025-03-25,11:32:55,SDU->BSB,250.29,1555.31,1500
1,2025-03-25,11:33:10,BSB->SSA,292.79,1819.40,1700
2,2025-03-25,11:33:33,"ORY->GIG, SSA->ORY",4415.09,27435.04,27000
3,2025-03-25,11:47:05,BSB->SSA,292.79,1819.40,1700
4,2025-03-25,11:47:28,"ORY->GIG, SSA->ORY",4415.09,27435.04,27000


In [7]:
# Create a proper datetime column by combining Date and Time columns
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Check the unique flights in the dataset
unique_flights = df['Flight'].unique()
print(f"Unique flights in the dataset: {unique_flights}")
print(f"Total number of unique flights: {len(unique_flights)}")

Unique flights in the dataset: ['SDU->BSB' 'BSB->SSA' 'ORY->GIG, SSA->ORY']
Total number of unique flights: 3


## Interactive Flight Price Trends with Plotly

Create an interactive plot using Plotly with datetime on the x-axis and Price_EUR on the y-axis. Each flight will be represented with a different color.

In [8]:
# Create an interactive plot with Plotly
fig = px.line(df, x='Datetime', y='Price_EUR', color='Flight',
              title='Flight Price Trends Over Time',
              labels={'Datetime': 'Date and Time', 'Price_EUR': 'Price (EUR)', 'Flight': 'Flight Route'},
              markers=True, line_shape='linear')

# Improve the layout
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Price (EUR)',
    legend_title='Flight Routes',
    height=600,
    width=1000,
    hovermode='closest',
    hoverlabel=dict(bgcolor="white", font_size=12),
    xaxis=dict(tickformat='%Y-%m-%d %H:%M'),
    title=dict(font=dict(size=20), x=0.5)
)

# Add hover data for better information display
fig.update_traces(hovertemplate='<b>%{fullData.name}</b><br>Date: %{x|%Y-%m-%d %H:%M}<br>Price: €%{y:.2f}')

# Show the plot
fig.show()

## Price Comparison with Target Prices

In [9]:
# Create a figure with subplots for each flight
fig = make_subplots(rows=len(unique_flights), cols=1,
                   subplot_titles=[f'Flight: {flight}' for flight in unique_flights],
                   shared_xaxes=True,
                   vertical_spacing=0.1)

# Add traces for each flight
for i, flight in enumerate(unique_flights):
    flight_data = df[df['Flight'] == flight]
    
    # Add line trace for price
    fig.add_trace(
        go.Scatter(x=flight_data['Datetime'], y=flight_data['Price_EUR'],
                  mode='lines+markers', name=f'{flight} - Price',
                  line=dict(width=2),
                  hovertemplate='Date: %{x|%Y-%m-%d %H:%M}<br>Price: €%{y:.2f}'),
        row=i+1, col=1
    )
    
    # Add horizontal line for target price
    target_price = flight_data['Target_Price'].iloc[0]
    fig.add_trace(
        go.Scatter(x=[flight_data['Datetime'].min(), flight_data['Datetime'].max()], 
                  y=[target_price, target_price], mode='lines',
                  name=f'{flight} - Target (€{target_price})',
                  line=dict(dash='dash', color='red'),
                  hoverinfo='skip'),
        row=i+1, col=1
    )
    
    # Add horizontal line for average price
    avg_price = flight_data['Price_EUR'].mean()
    fig.add_trace(
        go.Scatter(x=[flight_data['Datetime'].min(), flight_data['Datetime'].max()], 
                  y=[avg_price, avg_price], mode='lines',
                  name=f'{flight} - Avg (€{avg_price:.2f})',
                  line=dict(dash='dot', color='green'),
                  hoverinfo='skip'),
        row=i+1, col=1
    )

# Update layout
fig.update_layout(
    height=400*len(unique_flights),
    width=1000,
    title_text="Flight Price Trends vs Target Prices",
    showlegend=True,
    legend=dict(groupclick="toggleitem"),
    hovermode="closest",
    xaxis=dict(tickformat='%Y-%m-%d %H:%M')
)

# Update y-axes titles
for i in range(len(unique_flights)):
    fig.update_yaxes(title_text="Price (EUR)", row=i+1, col=1)

# Update x-axis title only for the bottom subplot
fig.update_xaxes(title_text="Date and Time", row=len(unique_flights), col=1)

# Show the plot
fig.show()

## Price Distribution Analysis

In [10]:
# Create separate histograms for each flight
fig = make_subplots(rows=1, cols=len(unique_flights),
                   subplot_titles=[f'Price Distribution: {flight}' for flight in unique_flights])

# Define colors for each flight
colors = px.colors.qualitative.D3

for i, flight in enumerate(unique_flights):
    flight_data = df[df['Flight'] == flight]
    target_price = flight_data['Target_Price'].iloc[0]
    
    # Add histogram trace
    fig.add_trace(
        go.Histogram(x=flight_data['Price_EUR'],
                    marker=dict(color=colors[i % len(colors)]),
                    opacity=0.7,
                    name=flight),
        row=1, col=i+1
    )
    
    # Add vertical line for target price
    fig.add_shape(type="line",
                  x0=target_price, y0=0, x1=target_price, y1=1,
                  xref=f"x{i+1}", yref=f"paper",
                  line=dict(color="red", width=2, dash="dash"))
    
    # Add annotation for target price
    fig.add_annotation(x=target_price, y=0.95,
                      text=f"Target: €{target_price}",
                      showarrow=True,
                      arrowhead=1,
                      xref=f"x{i+1}", yref="paper",
                      ax=0, ay=-40)

# Update layout
fig.update_layout(
    height=500,
    width=300*len(unique_flights),
    title_text="Price Distribution by Flight",
    showlegend=False,
    bargap=0.1
)

# Update axes titles
for i in range(1, len(unique_flights)+1):
    fig.update_xaxes(title_text="Price (EUR)", row=1, col=i)
    if i == 1:
        fig.update_yaxes(title_text="Frequency", row=1, col=i)

# Show the plot
fig.show()

## Daily Price Trends Analysis

In [11]:
# Calculate daily average prices
df['Date'] = pd.to_datetime(df['Date'])
daily_avg = df.groupby(['Date', 'Flight'])['Price_EUR'].mean().reset_index()

# Create interactive daily price trend chart
fig = px.line(daily_avg, x='Date', y='Price_EUR', color='Flight',
              title='Daily Average Flight Prices',
              labels={'Date': 'Date', 'Price_EUR': 'Average Daily Price (EUR)', 'Flight': 'Flight Route'},
              markers=True)

# Improve the layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Average Daily Price (EUR)',
    legend_title='Flight Routes',
    height=600,
    width=1000,
    hovermode='closest',
    hoverlabel=dict(bgcolor="white", font_size=12),
    title=dict(font=dict(size=20), x=0.5)
)

# Add hover information
fig.update_traces(hovertemplate='<b>%{fullData.name}</b><br>Date: %{x|%Y-%m-%d}<br>Avg Price: €%{y:.2f}')

# Show the plot
fig.show()

## Hourly Price Analysis

In [12]:
# Extract hour from datetime for analysis
df['Hour'] = df['Datetime'].dt.hour

# Calculate hourly average prices
hourly_avg = df.groupby(['Hour', 'Flight'])['Price_EUR'].mean().reset_index()

# Create interactive hourly price trend chart
fig = px.line(hourly_avg, x='Hour', y='Price_EUR', color='Flight',
              title='Average Flight Prices by Hour of Day',
              labels={'Hour': 'Hour of Day (24h)', 'Price_EUR': 'Average Price (EUR)', 'Flight': 'Flight Route'},
              markers=True)

# Improve the layout
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Show every hour
    xaxis_title='Hour of Day (24h)',
    yaxis_title='Average Price (EUR)',
    legend_title='Flight Routes',
    height=500,
    width=1000,
    hovermode='closest',
    title=dict(font=dict(size=20), x=0.5)
)

# Add hover information
fig.update_traces(hovertemplate='<b>%{fullData.name}</b><br>Hour: %{x}:00<br>Avg Price: €%{y:.2f}')

# Show the plot
fig.show()

## Price Change Over Time Analysis

In [13]:
# Calculate daily price changes
price_changes = {}

for flight in unique_flights:
    flight_data = df[df['Flight'] == flight].sort_values('Datetime')
    flight_data['Price_Change'] = flight_data['Price_EUR'].diff()
    flight_data['Price_Change_Pct'] = flight_data['Price_EUR'].pct_change() * 100
    price_changes[flight] = flight_data

# Create a dataframe with all price changes
price_change_df = pd.concat(price_changes.values())

# Create interactive price change chart
fig = px.scatter(price_change_df.dropna(), x='Datetime', y='Price_Change', color='Flight',
                 size=abs(price_change_df.dropna()['Price_Change']), size_max=15,
                 title='Price Changes Over Time',
                 labels={'Datetime': 'Date and Time', 'Price_Change': 'Price Change (EUR)', 'Flight': 'Flight Route'})

# Add zero line
fig.add_shape(type="line", x0=price_change_df['Datetime'].min(), y0=0, 
              x1=price_change_df['Datetime'].max(), y1=0,
              line=dict(color="gray", width=1, dash="dash"))

# Improve the layout
fig.update_layout(
    height=600,
    width=1000,
    hovermode='closest',
    hoverlabel=dict(bgcolor="white", font_size=12),
    xaxis=dict(tickformat='%Y-%m-%d %H:%M'),
    title=dict(font=dict(size=20), x=0.5)
)

# Customize hover information to show price change details
fig.update_traces(hovertemplate='<b>%{fullData.name}</b><br>Date: %{x|%Y-%m-%d %H:%M}<br>Price Change: €%{y:.2f}')

# Show the plot
fig.show()

## Interactive 3D Price Visualization

In [14]:
# Create a datetime index for consecutive numbers
df['Day_Number'] = (df['Datetime'].dt.date - df['Datetime'].dt.date.min()).dt.days

# Create 3D scatter plot
fig = px.scatter_3d(df, x='Day_Number', y='Hour', z='Price_EUR', color='Flight',
                    title='3D Visualization of Flight Prices Over Time',
                    labels={'Day_Number': 'Days (from first data point)', 
                           'Hour': 'Hour of Day', 
                           'Price_EUR': 'Price (EUR)', 
                           'Flight': 'Flight Route'})

# Improve the layout
fig.update_layout(
    height=800,
    width=1000,
    scene=dict(
        xaxis_title='Days (from first data point)',
        yaxis_title='Hour of Day',
        zaxis_title='Price (EUR)',
        xaxis=dict(nticks=10),
        yaxis=dict(nticks=24),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    title=dict(font=dict(size=20), x=0.5)
)

# Update hover template
fig.update_traces(hovertemplate='<b>%{fullData.name}</b><br>Day: %{x}<br>Hour: %{y}:00<br>Price: €%{z:.2f}')

# Show the plot
fig.show()

AttributeError: Can only use .dt accessor with datetimelike values

## Summary Statistics

In [ ]:
# Calculate comprehensive statistics for each flight
flight_stats = df.groupby('Flight').agg(
    min_price=('Price_EUR', 'min'),
    max_price=('Price_EUR', 'max'),
    avg_price=('Price_EUR', 'mean'),
    median_price=('Price_EUR', 'median'),
    std_price=('Price_EUR', 'std'),
    price_range=('Price_EUR', lambda x: x.max() - x.min()),
    records=('Price_EUR', 'count'),
    first_date=('Datetime', 'min'),
    last_date=('Datetime', 'max')
).reset_index()

# Add target price and difference columns
flight_stats = flight_stats.merge(
    df[['Flight', 'Target_Price']].drop_duplicates(),
    on='Flight'
)

flight_stats['diff_from_target'] = flight_stats['avg_price'] - flight_stats['Target_Price']
flight_stats['pct_diff'] = (flight_stats['diff_from_target'] / flight_stats['Target_Price']) * 100
flight_stats['days_monitored'] = (flight_stats['last_date'] - flight_stats['first_date']).dt.days + 1

# Display the statistics in a formatted table using Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=['Flight', 'Min Price (€)', 'Max Price (€)', 'Avg Price (€)', 
                        'Target Price (€)', 'Diff from Target (€)', 'Diff %', 'Days Monitored'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[flight_stats['Flight'], 
                      flight_stats['min_price'].round(2), 
                      flight_stats['max_price'].round(2), 
                      flight_stats['avg_price'].round(2),
                      flight_stats['Target_Price'].round(2),
                      flight_stats['diff_from_target'].round(2),
                      flight_stats['pct_diff'].round(2),
                      flight_stats['days_monitored']],
               fill_color=['white', 
                          ['lightgreen' if val <= target else 'lightcoral' 
                           for val, target in zip(flight_stats['min_price'], flight_stats['Target_Price'])],
                          'white',
                          ['lightgreen' if val <= target else 'lightcoral' 
                           for val, target in zip(flight_stats['avg_price'], flight_stats['Target_Price'])],
                          'white',
                          ['lightgreen' if val <= 0 else 'lightcoral' 
                           for val in flight_stats['diff_from_target']],
                          ['lightgreen' if val <= 0 else 'lightcoral' 
                           for val in flight_stats['pct_diff']],
                          'white'],
               align='left',
               format=[None, '.2f', '.2f', '.2f', '.2f', '.2f', '.2f%', None]))
])

fig.update_layout(
    title='Flight Price Statistics Summary',
    height=150 + 50*len(flight_stats),
    width=1000
)

fig.show()

## Summary and Conclusion

This analysis provides interactive visualizations of flight price trends over time. Key features include:

1. **Interactive time series** showing exact price movements with precise datetime on the x-axis
2. **Price comparison against targets** with clear visual indicators
3. **Distribution analysis** to understand the price ranges and frequencies
4. **Time-based patterns** including daily averages and hourly trends
5. **Price change monitoring** to detect significant price movements
6. **3D visualization** offering a unique perspective on price variations across time dimensions

The interactive nature of these Plotly visualizations allows for deeper exploration by hovering, zooming, and filtering to uncover insights that might help in making optimal flight booking decisions.